# Getting started with Flask

Flask is a framework that lets you develop your own web apps. Today, we
are going to use it to create an API that we can then test locally and
later publish to the cloud!

In order to install Flask, you need to use `pip install flask jsonify`.
With this command, we are also installing jsonify, which helps convert
objects to JSON data.

## The app

Taking in the requirements is a crucial step when you start developing
your application. In our case, the requirements aren't many. Contoso Books Ltd would like us to:  
- Develop a book store app
- Users need to list all books, search for books and access books via ISBN
- Users should be able to create new books, updating is not required
- Users should be able to delete books
- Books consist of isbn, author and name. Optionally, they may have a description.

Think a moment about what this means for the methods our app needs to support.

Thought about it? Great. Listing all books sounds just like GET, while creating
new books would rather be POST. Deleting a book can be achieved with DELETE.

Since the customer did not specify any other purposes for their store, we
can assume that we only need one API endpoint: Books! Books are the only
type of resource we store here.

This means, our URIs will look like `https://contoso.books/Books` and 
`https://contoso.books/Books/1473200261`. Since the user needs to be able
to search for books, the URIs will need to contain parameters, which we
have gotten to know as Query Strings.
So, a URI might also look like `https://contoso.books/Books?author=Terry%20Pratchett`.

Since we need to create books, we not only need to take in qery strings
but also a JSON body.

## Getting started

First of all, we need to import flask and create a list of books. In a real scenario
the list of books would be stored in a relational database! Here, for the
sake of simplicity, we use a list. This of course means that after a restart, all books
are reset.

In [ ]:
from flask import Flask
# The application name is set to the name of the module
app = Flask(__name__)

# We are using a static book store. Nothing fancy
books = [
    {
        'isbn': '1473200261',
        'name': 'Witches abroad',
        'author': 'Terry Pratchett'
    },
    {
        'isbn': '006051518X',
        'name': 'Anansi boys',
        'author': 'Neil Gaiman'
    }
]

## GET

The first method we will implement is our basic GET - list books. In an API, we define what is
called a route. The route is relative to your application and contains the resource

In [ ]:
# Routes define your API endpoints, if you will
# @app.route() is called a Decorator - it "decorates" a function
# This route is a dynamic route that can contain the ISBN of a book
@app.route("/Books", methods=['GET'])
@app.route("/Books/<isbn>", methods=['GET'])
def get_books(isbn=None):
    books_to_return = books.copy()

    if isbn:
        books_to_return = [book for book in books if book['isbn'] == isbn]

    if 'author' in request.args:
        books_to_return = [
            book for book in books if book['author'] == request.args['author']]

    if len(books_to_return) == 0:
        response = app.response_class(
            status=404
        )
    else:
        response = app.response_class(
            response=json.dumps(books_to_return),
            status=200,
            mimetype='application/json'
        )

    return response

This app can be executed as is in order to try it locally. To do this, change into the directory of your
application, and execute `flask run`.

Flask will continue to host your app. Had you executed your script directly with Python, nothing
would have happened apart from you defining a list and a function!



In [ ]:
import requests

response = requests.get('http://localhost:5000/Books')
for book in response.json():
  print('Found book ' + book['name'] + ' by ' + book['author'] + ' in our library')

# POST

This method accepts a JSON body which it derives the data from, and checks the book store first.
If the book to be added already exists, it returns an error 409 (conflict).

If the add was successful, the API returns 204 (success, but no output).

In [ ]:
# The same route is now added with a POST method
# We want new books to be added and use the request body to send this data
# The entire request is stored in the request variable, and the body
# is contained in request.data
@app.route("/Books", methods=['POST'])
def add_books():
    try:
        request_data = request.json
        print('req ok')
        new_book = {
            'isbn': request_data['isbn'],
            'name': request_data['name'],
            'author': request_data['author']
        }
        
        if any(book.get('isbn', None) == new_book['isbn'] for book in books):
            return app.response_class(
                response= f'Uh oh... book with ISBN {new_book["isbn"]} already exists',
                status=409
            )

        books.append(new_book)
        return app.response_class(
            status=204
        )
    except:
        return app.response_class(
            response='Uh oh... This was not supposed to happen :(',
            status=404
        )

### DELETE

The DELETE route is rather simple. It has to be called with the ISBN of the book in the URI
and will remove that book. To allow passing the book as a query string or in the body,
you would simply add a route that does not require an ISBN and do the same checks as you
have done in your GET route.

In [ ]:
# This route is a dynamic route that can contain the ISBN of a book
@app.route("/Books/<isbn>", methods=['DELETE'])
def delete_book(isbn):
    filtered_books = [book for book in books if book['isbn'] != isbn]
    books.clear()
    for book in filtered_books:
        books.append(book)
    return app.response_class(
        status=204
    )

### Publishing the final app

To make use of the cloud, you will need either a paid subscription by your school or employer, or you
can make use of the free trial periods available from all big providers:
- Amazon Web Services: https://aws.amazon.com/free
- Microsoft Azure: https://azure.microsoft.com/en-us/free
- Google Gloud Project: https://cloud.google.com/free

Depending on the provider, you can provision a resource to host your web app.

Going a slightly more traditional road, you can of course always host your web apps yourself.
This is usually done in what is called a LAMP stack (Linux Apache MariaDb Python).
On Windows, the XAMPP project provides such a stack for Windows as well.

To you, as a developer, the operating system should hardly matter. Which is one benefit
of going the PaaS route with one of the cloud providers - everything apart from
your app is taken care of. Installation, Configuration, Updates, Security - the
only thing you need to do is develop and publish!

### Some helpful online courses

To work a bit more with the cloud and make the most of your free trials with the different
providers, check out these free courses on Microsoft Learn which will hone
your Python skills as well:
- https://docs.microsoft.com/en-us/learn/paths/foundations-data-science/
- https://docs.microsoft.com/en-us/learn/modules/python-flask-build-ai-web-app/
- https://docs.microsoft.com/en-us/learn/paths/python-first-steps/
- https://docs.microsoft.com/en-us/learn/paths/introduction-python-space-exploration-nasa/

All of the learning paths linked above can be done without any cloud subscription,
you only need Python and an editor of your choice!
